### TODO:: 

1. build date function and convert utc to mm dd - done!!!

    1.1 - instead of using date select miliseconds as test data from 11.2018 / check if at least 20% of data


2. remove duplicate items per single person / remove by bidid  - done my michael - done!!!


3. remove trash columns from android_bids

    3.1 remove marketplace col - done !!!
    
    3.2 device_maker and device_model NaN's changed to unknown and claasified as new device_maker or device_model - done!!!
    
    

4. remove trash columns from play_apps

    4.1 Create dataframe from files - done!!!
    
    4.2 Do we need to use play_apps -> Which cols do we want to use??
    
        [video, category, score, histogram, reviews, editors_choice, (price, free), iap, size, installs, content_rating, iap_range, interactive_elements?, bids]


5. trun into comma seperated label for play_apps -> category


6. Merge data from two tables android_bids_us and play_apps

    6.1 USE pandas merge datafreames function to merge both files

In [ ]:
#imports
import pandas as pd
from datetime import datetime
from zipfile import ZipFile
import pickle
from pathlib import Path


In [ ]:
#Read orig android data
origdata = pd.read_csv('data/android_bids_us.csv')
origdata.head()


In [ ]:
#Read orig play_apps data
app_details_orig = ZipFile("data/play_apps.zip")
app_file = 'play_apps/a008.com.fc2.blog.androidkaihatu.datecamera2'
app = pickle.loads(app_details_orig.read(app_file))
app

# Analyze android dataset

In [ ]:
dataTypes = origdata.dtypes
print(f'Total columns: {dataTypes.size} ||| Total rows: {origdata.count()}')
print(f'Data type of each column of Dataframe : ')
print(dataTypes)
# We may have NAN's in user_isp, device_maker, device_model

print('-----------------------------------')
print('Amount of NaN under utc_time', origdata['utc_time'].isnull().sum())
# all items in marketplace col equal 'chartboost'. we can remove col

#Print start time 
start_time = origdata['utc_time'].min()
end_time = origdata['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(start_time/1000))
print('End Time: ',  datetime.fromtimestamp(end_time/1000))

print('-----------------------------------')


print('-----------------------------------')
print(origdata.marketplace.unique())
# all items in marketplace col equal 'chartboost'. we can remove col
print('-----------------------------------')

print('-----------------------------------')
print('app_id::')
print('Amount of NaN under app_id', origdata['app_id'].isnull().sum())
print(origdata.app_id.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_ip::')
print(origdata.user_isp.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_state::')
print(origdata.user_state.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('device_maker::')
print(origdata.device_maker.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('device_model::')
print(origdata.device_model.value_counts())
print('-----------------------------------')



# Prepare android dataset

In [ ]:
#Rearrange dataframe
#clone orig dataframe
android_df = origdata.copy()

#-----------------------------

#Add date from utc and remove utc
#android_df['date'] = android_df['utc_time'].apply(lambda x: datetime.fromtimestamp(x/1000))
#android_df = android_df.drop(['utc_time'], axis=1)

#-----------------------------

#Remove marketplace col
android_df = android_df.drop(['marketplace'], axis=1)

#-----------------------------

#Remove duplicated bidid. Keep only single instance keep the one with click == 1
android_df = android_df[~((android_df['bidid'].duplicated(keep=False))&(android_df['click']==0))]
#and then remove bids column
android_df = android_df.drop(['bidid'], axis=1)

#-----------------------------

#Change device_maker NaN's to unknown manufacturerer
android_df['device_maker'] = android_df['device_maker'].apply(lambda x: 'unknown' if x!=x else x)

#-----------------------------

#Change device_model NaN's to unknown valid model
android_df['device_model'] = android_df['device_model'].apply(lambda x: 'unknown' if x!=x else x)

#-----------------------------

android_df

# Extract play_apps data and first clean

In [ ]:
# Get play_apps data

def zipToDataFrame(zipped, to_remove):
    rows = []
    
    for f in zipped.filelist:
        app_file = f.filename
        try:
            app = pickle.loads(zipped.read(app_file))
        except EOFError as e:
            continue
        except pickle.UnpicklingError as e:
            continue
        except Exception as e:
            return
            
        # Remove columns
        for col in to_remove:
            del app[col]
        rows.append(app)
    
    return pd.DataFrame(rows)


app_details = ZipFile("data/play_apps.zip")
cols_to_remove = ['title','screenshots', 'icon', 
                  'histogram', 'description', 'description_html', 
                  'recent_changes', 'developer', 'developer_id', 'developer_email', 'developer_url',
                  'developer_address', 'url', 'current_version', 'updated', 'required_android_version']


orig_play_apps = zipToDataFrame(app_details, cols_to_remove)
orig_play_apps



# Analyze play_apps dataset

In [ ]:
# explore play_apps data characteristics

apps_dataTypes = orig_play_apps.dtypes

print(f'Total columns: {apps_dataTypes.size} ||| Total rows: {orig_play_apps.count()}')
print(f'Data type of each column of Dataframe : ')
print(apps_dataTypes)
# We may have NAN's in iap_range, interactive_elements, price

print('-----------------------------------')
print('price values: ', orig_play_apps.price.unique())
print('free values: ',orig_play_apps.free.unique())
# items in price and free should be in a single col
print('-----------------------------------')

print('-----------------------------------')
#print('iap_range: ', orig_play_apps.iap_range.unique())
# items in price and free should be in a single col
print('-----------------------------------')



# Prepare play_apps dataset

In [ ]:
#Fix and manipulate play_apps data before merge with android_bids_us
#clone orig dataframe
play_apps_df = orig_play_apps.copy()

# Handle 'video' column
play_apps_df['video'] = play_apps_df['video'].apply(lambda x: 0 if pd.isna(x) else 1)
#---------------------------------------------------

play_apps_df



# Merge two datasets

In [ ]:
print('---------------------------------------------')
#merge two data sets into merged dataframe
merged_df = pd.merge(android_df, play_apps_df, on='app_id')
merged_df = merged_df.drop(['app_id'], axis=1)

print('Rows before merge: ',android_df.shape[0])
print('Rows after merge: ', merged_df.shape[0])

#Concolusion after merge
# from android = 2936921 rows × 10 columns turn into 2656577 rows × 24 columns
# we missed ~300k rows. need to check y.....
print('---------------------------------------------')


merged_df




# Split data and save to files

In [ ]:
print('---------------------------------------------')
print('Amount of NaN under utc_time: ', merged_df['utc_time'].isnull().sum())
print('Amount of NaN in all data : ', merged_df.isnull().sum().sum())
print('---------------------------------------------')


print('---------------------------------------------')
#Print start time after merge
merged_start_time = merged_df['utc_time'].min()
merged_end_time = merged_df['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(merged_start_time/1000))
print('End Time: ',  datetime.fromtimestamp(merged_end_time/1000))
print('Split time: ',  datetime.fromtimestamp(1541023200000/1000))
print('---------------------------------------------')


print('---------------------------------------------')
tarin_df, test_df = merged_df[(mask:=merged_df['utc_time'] < 1541023200000)], merged_df[~mask]

tarin_df = tarin_df.drop(['utc_time'], axis=1)
test_df = test_df.drop(['utc_time'], axis=1)

tarin_size = tarin_df.shape[0]
test_size = test_df.shape[0]

print('Amount of tarin_df data : ', tarin_size)
print('Amount of test_df data : ', test_size)

print(f'Test data is: {(test_size / (tarin_size + test_size))*100} percent')

      
print('---------------------------------------------')
      
#Save to CSV files
#Save to splited csv files
filepath1 = Path('data/train.csv')  
filepath2 = Path('data/test.csv')  
tarin_df.to_csv(filepath1, index=False)
test_df.to_csv(filepath2, index=False)

